In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers,models
from tensorflow.keras.callbacks import EarlyStopping
import os
import numpy as np
import random

In [2]:
img_height,img_width = 128,128
batch_size = 32

test_datagen  = ImageDataGenerator(rescale = 1./255)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.7,1.3],
    validation_split=0.2
)


val_datagen  = ImageDataGenerator(rescale = 1./255)

In [3]:
train_generator = train_datagen.flow_from_directory(
    'painting_dataset/train',
    target_size=(img_height,img_width),
    batch_size = batch_size,
    class_mode = 'binary'
)

test_generator = test_datagen.flow_from_directory(
    'painting_dataset/test',
    target_size=(128,128),
    batch_size=32,
    class_mode='binary',
    shuffle=False  
)
val_generator = val_datagen.flow_from_directory(
    'painting_dataset/val',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

Found 3126 images belonging to 2 classes.
Found 348 images belonging to 2 classes.
Found 348 images belonging to 2 classes.


In [4]:
model = models.Sequential([
    layers.Conv2D(32,(3,3),activation="relu",input_shape=(img_height,img_width,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation="relu"),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128,(3,3),activation="relu"),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1,activation="sigmoid")
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [5]:
model.fit(train_generator,validation_data = val_generator,epochs = 10,callbacks=[early_stop])

Epoch 1/10
98/98 [==============================] - 22s 216ms/step - loss: 0.6512 - accuracy: 0.6424 - val_loss: 0.5076 - val_accuracy: 0.7414
Epoch 2/10
98/98 [==============================] - 22s 219ms/step - loss: 0.6172 - accuracy: 0.6660 - val_loss: 0.5624 - val_accuracy: 0.6954
Epoch 3/10
98/98 [==============================] - 21s 218ms/step - loss: 0.6075 - accuracy: 0.6916 - val_loss: 0.4703 - val_accuracy: 0.8132
Epoch 4/10
98/98 [==============================] - 22s 221ms/step - loss: 0.5806 - accuracy: 0.7095 - val_loss: 0.5059 - val_accuracy: 0.7270
Epoch 5/10
98/98 [==============================] - 23s 232ms/step - loss: 0.5659 - accuracy: 0.7230 - val_loss: 0.5295 - val_accuracy: 0.7328
Epoch 6/10
98/98 [==============================] - 23s 231ms/step - loss: 0.5585 - accuracy: 0.7300 - val_loss: 0.5264 - val_accuracy: 0.7011


In [6]:
test_loss, test_acc = model.evaluate(test_generator)

print("Test Accuracy:", test_acc)
print("Test Loss:", test_loss)

11/11 [==============================] - 1s 43ms/step - loss: 0.5259 - accuracy: 0.7529
Test Accuracy: 0.7528735399246216
Test Loss: 0.5258991718292236


In [13]:

test_folder = "painting_dataset/test"

# Collect all test images
all_images = []

for root, dirs, files in os.walk(test_folder):
    for file in files:
        if file.lower().endswith((".png", ".jpg", ".jpeg")):
            all_images.append(os.path.join(root, file))

# Randomly choose image
img_path = random.choice(all_images)

print("Testing Image:", img_path)

# Load and preprocess image
img = image.load_img(img_path, target_size=(128,128))
img_array = image.img_to_array(img)/255.0
img_array = np.expand_dims(img_array, axis=0)

# Prediction
prediction = model.predict(img_array)

if prediction[0][0] > 0.5:
    print("Prediction : Normal Painting")
else:
    print("Prediction : Hidden Painting")

Testing Image: painting_dataset/test\no_hidden\t3_l5f9bw.jpg
1/1 [==============================] - 0s 24ms/step
Prediction : Normal Painting


In [11]:
model.save("hidden_image_detector.h5")

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
